In [1]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
from random import randint
import random
import time
import os
from unidecode import unidecode
from fuzzywuzzy import process

import os
import zipfile
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
from random import randint
import random
import time
import os
import fiona
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import Levenshtein

In [4]:
df_R1_matricula = pd.read_csv("../../230628_COMPARACION_R1_DIFERENTES_FUENTES/output/230703_R1_Consolidado_4_mayo_Matricula.csv", encoding = "latin1")
df_R1_matricula["CODIGO_ORIP"] = df_R1_matricula["CODIGO_ORIP"].astype(str).str.replace(".0", "")

df_R1_matricula["MUNICIPIO"] = df_R1_matricula["MUNICIPIO"].astype(int).apply(round).astype(str).str.zfill(3)
df_R1_matricula["DEPARTAMENTO"] = df_R1_matricula["DEPARTAMENTO"].astype(int).apply(round).astype(str).str.zfill(2)
df_R1_matricula["DIVIPOLA"] = df_R1_matricula["DEPARTAMENTO"] + df_R1_matricula["MUNICIPIO"]
#ofertas_final["Municipio_x"] = ofertas_final["Municipio_x"].astype(int).apply(round).astype(str).str.zfill(3)
#ofertas_final["Departamento_x"] = ofertas_final["Departamento_x"].astype(int).apply(round).astype(str).str.zfill(2)

C:\Users\Administrador\AppData\Local\Temp\ipykernel_7416\3658018204.py:1: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_R1_matricula = pd.read_csv("../../230628_COMPARACION_R1_DIFERENTES_FUENTES/output/230703_R1_Consolidado_4_mayo_Matricula.csv", encoding = "latin1")


In [5]:
# df_matricula_R1 = pd.read_csv("../input/20230526_R1_Matricula.csv")
# df_matricula_R1_v2 = df_matricula_R1[df_matricula_R1["MATRICULA_INMOBILIARIA"].notnull()][["NPN", "MATRICULA_INMOBILIARIA", "NUMERO_DEL_PREDIO", 'Departamento', 'Municipio', "DEPARTAMENTO", "DIVIPOLA"]]
# df_matricula_R1_v2 = df_matricula_R1_v2.rename(columns={"NPN" : "NPN_R1", "MATRICULA_INMOBILIARIA" : "MATRICULA_INMOBILIARIA_R1", "NUMERO_DEL_PREDIO" : "NUMERO_DEL_PREDIO_R1", 'Departamento' : 'Departamento_R1', 'Municipio' : 'Municipio_R1', "DEPARTAMENTO" : "CODIGO_DEPARTAMENTO_R1", "DIVIPOLA" : "DIVIPOLA_R1"})
# df_matricula_R1_v2.columns
# #df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1_v2"] = df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1"].str.strip()
# df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1"] = df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1"].str.strip()
# pattern = r'[^a-zA-Z0-9]'

#df_matricula_R1_v2['MATRICULA_INMOBILIARIA_R1_v2'] = df_matricula_R1_v2['MATRICULA_INMOBILIARIA_R1_v2'].astype(str).str.replace(pattern, '', regex = True)

# df_matricula_R1_v3 = df_matricula_R1_v2[df_matricula_R1_v2['MATRICULA_INMOBILIARIA_R1'].str.count("-") == 1]
# df_matricula_R1_v3['Matricula'] = df_matricula_R1_v3['MATRICULA_INMOBILIARIA_R1'].str.extract(r'-(\d+)')
# df_matricula_R1_v3['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_matricula_R1_v3['DIVIPOLA_R1'].astype(int).apply(round).astype(str).str.strip() + "-" + df_matricula_R1_v3['Matricula'].astype(str).str.strip()

In [6]:
ofertas_final = pd.read_csv("../output/230712_RESULTADOS_GEOREF.csv")
ofertas_final["DIVIPOLA"] = ofertas_final["DIVIPOLA"].fillna(0)
ofertas_final["DIVIPOLA"] = ofertas_final["DIVIPOLA"].astype(int).apply(round).astype(str).str.zfill(5)

C:\Users\Administrador\AppData\Local\Temp\ipykernel_7416\1282143781.py:1: DtypeWarning: Columns (5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,32,33,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  ofertas_final = pd.read_csv("../output/230712_RESULTADOS_GEOREF.csv")


In [7]:
tabla_codigos_orip = pd.read_excel("../input/tablaORIP.xlsx", sheet_name = "Hoja1")

#tabla_codigos_orip["CÓDIGO ORIP"] = tabla_codigos_orip["CÓDIGO ORIP"].str.strip().astype(str)
tabla_codigos_orip["DIVIPOLA"] = tabla_codigos_orip["DIVIPOLA"].fillna(0)
tabla_codigos_orip["DIVIPOLA"] = tabla_codigos_orip["DIVIPOLA"].astype(str).str.replace(".0", "").str.zfill(5)
tabla_codigos_orip[tabla_codigos_orip["DIVIPOLA"].astype(str).str.contains("25168")]

,CÓDIGO ORIP,DEPARTAMENTO,COBERTURA,DIVIPOLA,NOMBRE OFICINA DE REGISTRO,NOMBRE ORIP,FECHA DE CREACIÓN,ACTO ADMINISTRATIVO DE CREACIÓN,JEFE DE OFICINA,CORREO JEFE,CONTACTO
603,162,CUNDINAMARCA,CHAGUANÍ,25168,GUADUAS,NaN,NaN,NaN,DAVID RAMIREZ GUERRA,david.ramirez@supernotariado.gov.co,(1) 8417430 (1)8466266


In [8]:
codigos_orip_divipola = tabla_codigos_orip[["DIVIPOLA", "CÓDIGO ORIP"]].drop_duplicates("DIVIPOLA")
df_R1_matricula["DIVIPOLA"] = df_R1_matricula["DIVIPOLA"].astype(str)
ofertas_final["DIVIPOLA"] = ofertas_final["DIVIPOLA"].astype(str)
codigos_orip_divipola["DIVIPOLA"] = codigos_orip_divipola["DIVIPOLA"].astype(str)
df_R1_matricula = pd.merge(df_R1_matricula, codigos_orip_divipola, how = "left", on = "DIVIPOLA")
ofertas_final = pd.merge(ofertas_final, codigos_orip_divipola, how = "left", on = "DIVIPOLA")

In [9]:
df_matriculas = pd.DataFrame({"Matricula_Inmobiliaria" : ofertas_final["Matricula_Inmobiliaria"].str.strip().str.extractall(r'(\d+)')[0].reset_index(level=1, drop=True)})
df_matriculas.tail()

df_merge = pd.merge(df_matriculas, ofertas_final[["CÓDIGO ORIP"]], how = "left", left_index = True, right_index = True)

df_merge["Matricula_Divipola"] = df_merge["CÓDIGO ORIP"].astype(str) + "-" + df_merge["Matricula_Inmobiliaria"].astype(str)
#ofertas_final[["DIVIPOLA"]]

In [10]:
df_R1_matricula["MATRICULA"].fillna(0, inplace=True)
df_R1_matricula = df_R1_matricula[df_R1_matricula["NUMERO_DE_ORDEN"].isin([1, "1"])]
df_R1_matricula["MATRICULA"] = df_R1_matricula["MATRICULA"].astype(str).str.replace(".0", "").str.strip()
df_R1_matricula["NPN_R1"] = df_R1_matricula["DEPARTAMENTO"] + df_R1_matricula["MUNICIPIO"] + df_R1_matricula["NUMERO_DEL_PREDIO"]
#df_R1_matricula['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_R1_matricula['CÓDIGO ORIP'].astype(str).str.strip() + "-" + df_R1_matricula['MATRICULA'].astype(str).str.strip()
#df_R1_matricula['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_R1_matricula['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'].str.strip().str.replace(" ", "").astype(str)
df_merge['Matricula_Divipola'] = df_merge['Matricula_Divipola'].str.strip().str.replace(" ", "").astype(str)
df_merge.reset_index(inplace= True)
df_merge_2 = pd.merge(df_merge, df_R1_matricula[["CÓDIGO ORIP", "MATRICULA", "DEPARTAMENTO", "MUNICIPIO", "NUMERO_DEL_PREDIO", "NPN_R1"]].drop_duplicates(), how = "inner", left_on = ["CÓDIGO ORIP", "Matricula_Inmobiliaria"], right_on = ["CÓDIGO ORIP", "MATRICULA"])
df_merge_2.drop_duplicates("index",inplace= True)
#df_merge_2.set_index("index", inplace= True)
df_merge_2.shape
	

(32720, 9)

In [11]:
ORIP_UNIQUE_CODES = df_merge[df_merge["CÓDIGO ORIP"].isin(df_R1_matricula["CÓDIGO ORIP"])]["CÓDIGO ORIP"].unique()

df_aprox_matriculas_final = []
for orip in ORIP_UNIQUE_CODES:
    orip = ORIP_UNIQUE_CODES[0]
    df_avaluos = df_merge[df_merge["CÓDIGO ORIP"].astype(str) == str(orip)].copy()
    df_matricula_orip = df_R1_matricula[df_R1_matricula["CÓDIGO ORIP"].astype(str) == str(orip)].copy()

    df_avaluos = df_avaluos[~df_avaluos["Matricula_Inmobiliaria"].str.contains("\D", regex = True)]
    df_matricula_orip = df_matricula_orip[["CÓDIGO ORIP", "MATRICULA", "DEPARTAMENTO", "MUNICIPIO", "NUMERO_DEL_PREDIO", "NPN_R1"]].drop_duplicates()
    df_matricula_orip = df_matricula_orip[~df_matricula_orip["MATRICULA"].str.contains("\D", regex = True)]

    #df_avaluos["Matricula_Inmobiliaria"] = df_avaluos["Matricula_Inmobiliaria"].astype(int)

    #df_avaluos.sort_values("Matricula_Inmobiliaria")
    #df_avaluos
    #df_matricula_orip[["MATRICULA"]]
    df_avaluos["Matricula_Inmobiliaria"] = df_avaluos["Matricula_Inmobiliaria"].astype("Int64")
    df_matricula_orip["MATRICULA"] = df_matricula_orip["MATRICULA"].astype("Int64")
    df_avaluos["Matricula_Inmobiliaria"] = df_avaluos["Matricula_Inmobiliaria"].astype("int")
    df_matricula_orip["MATRICULA"] = df_matricula_orip["MATRICULA"].astype("int")
    df_avaluos.sort_values("Matricula_Inmobiliaria", inplace=True)
    df_matricula_orip.sort_values("MATRICULA", inplace=True)

    df_avaluos_matricula_aprox = pd.merge_asof(df_avaluos, df_matricula_orip[["MATRICULA","NPN_R1"]], left_on = "Matricula_Inmobiliaria", right_on = "MATRICULA")

    # Function to calculate Levenshtein distance
    def calculate_distance(row):
        return Levenshtein.distance(row['Matricula_Inmobiliaria'], row['MATRICULA'])

    df_avaluos_matricula_aprox["Matricula_Inmobiliaria"] = df_avaluos_matricula_aprox["Matricula_Inmobiliaria"].astype(str)
    df_avaluos_matricula_aprox["MATRICULA"] = df_avaluos_matricula_aprox["MATRICULA"].astype(str)
    # Apply the function to each row and store the results in a new column
    df_avaluos_matricula_aprox['distance'] = df_avaluos_matricula_aprox.apply(calculate_distance, axis=1)
    df_avaluos_matricula_aprox['Longitud_matricula'] = df_avaluos_matricula_aprox["Matricula_Inmobiliaria"].apply(len)
    df_aprox_matriculas_final.append(df_avaluos_matricula_aprox)
    #df_avaluos_matricula_aprox.tail()


In [12]:
df_aprox_matriculas_final_2 = pd.concat(df_aprox_matriculas_final, axis = 0)
df_aprox_matriculas_final_3 = df_aprox_matriculas_final_2[(df_aprox_matriculas_final_2["distance"] >= 1) & (df_aprox_matriculas_final_2["distance"] < 2) & (df_aprox_matriculas_final_2["Longitud_matricula"] > 4)]
df_aprox_matriculas_final_3["MATRICULA_APROXIMADA"] = df_aprox_matriculas_final_3["CÓDIGO ORIP"].astype(str) + "-" + df_aprox_matriculas_final_3["MATRICULA"].astype(str)
df_aprox_matriculas_final_4 = df_aprox_matriculas_final_3[["index", "MATRICULA_APROXIMADA", "NPN_R1"]]
df_aprox_matriculas_final_4.rename({"NPN_R1": "NPN_R1_APROX"}, axis = 1, inplace=True)
df_aprox_matriculas_final_4.set_index("index", inplace=True)
df_aprox_matriculas_final_4.head()


C:\Users\Administrador\AppData\Local\Temp\ipykernel_7416\4173479554.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aprox_matriculas_final_3["MATRICULA_APROXIMADA"] = df_aprox_matriculas_final_3["CÓDIGO ORIP"].astype(str) + "-" + df_aprox_matriculas_final_3["MATRICULA"].astype(str)
C:\Users\Administrador\AppData\Local\Temp\ipykernel_7416\4173479554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aprox_matriculas_final_4.rename({"NPN_R1": "NPN_R1_APROX"}, axis = 1, inplace=True)


,MATRICULA_APROXIMADA,NPN_R1_APROX
index,,
12922,200-10126,413570000000000130035000000000
483172,200-13461,410200101000000230015000000000
7062,200-13550,410200400000000080001000000000
24553,200-14431,413570000000000100044000000000
7194,200-15522,411320000000000330032000000000


In [13]:
df_aprox_matriculas_final_3.head()

,index,Matricula_Inmobiliaria,CÓDIGO ORIP,Matricula_Divipola,MATRICULA,NPN_R1,distance,Longitud_matricula,MATRICULA_APROXIMADA
511,12922,10127,200,200-10127,10126,413570000000000130035000000000,1,5,200-10126
521,483172,13466,200,200-00013466,13461,410200101000000230015000000000,1,5,200-13461
522,7062,13555,200,200-13555,13550,410200400000000080001000000000,1,5,200-13550
525,24553,14432,200,200-14432,14431,413570000000000100044000000000,1,5,200-14431
530,7194,15529,200,200-15529,15522,411320000000000330032000000000,1,5,200-15522


In [14]:
df_merge_2.set_index("index", inplace = True)
df_merge_2["FUENTE"] = "R1_MATRICULA"

In [15]:
ofertas_final_v1 = pd.merge(ofertas_final, df_merge_2[["NPN_R1", "FUENTE"]], how = "left", left_index = True, right_index = True)
ofertas_final_v2 = pd.merge(ofertas_final_v1,  df_aprox_matriculas_final_4, how = "left", left_index = True, right_index = True)
df_R1_matricula.head()
#pd.merge

,DEPARTAMENTO,MUNICIPIO,NUMERO_DEL_PREDIO,NUMERO_DE_ORDEN,TOTAL_REGISTROS,DIRECCION,COMUNA,DESTINO_ECONOMICO,AREA_TERRENO,AREA_CONSTRUIDA,AVALUO,VIGENCIA,NUMERO_PREDIAL_ANTERIOR,MATRICULA_INMOBILIARIA,CODIGO_ORIP,MATRICULA,DIVIPOLA,CÓDIGO ORIP,NPN_R1
0,08,078,0001000000000001000000000,1,1,SAN RAFAEL,NaN,D,210000.0,289,1.408150e+08,1012023,1.000000e+11,040-624,040,624,08078,40,080780001000000000001000000000
1,08,078,0001000000000002000000000,1,5,SAN BLAS,NaN,D,1183240.0,479,7.866300e+08,1012023,1.000000e+11,040-164824,040,164824,08078,40,080780001000000000002000000000
6,08,078,0001000000000003000000000,1,1,SAN BLAS,NaN,D,175000.0,0,1.877340e+08,1012023,1.000000e+11,12P003700306580000,12P003700306580000,0,08078,40,080780001000000000003000000000
7,08,078,0001000000000004000000000,1,1,CASA DE PIEDRA,NaN,A,327236.0,32,5.058330e+08,1012023,1.000000e+11,040-142551,040,142551,08078,40,080780001000000000004000000000
8,08,078,0001000000000005000000000,1,2,LA GRANJA,NaN,D,1140000.0,417,1.110397e+09,1012023,1.000000e+11,040-240435,040,240435,08078,40,080780001000000000005000000000


In [16]:
quantiles = list(np.arange(0, 0.95, 0.05))#[0.05, 0.1, 0.2, 0.25, 0.50, 0.75, 0.90, 0.95, 0.97]
ofertas_final_v2[(ofertas_final_v2["ZONA"] == "RURAL_APROX") & (ofertas_final_v2["FUENTE"] == "R1_MATRICULA")]["Distancia_final"].quantile(quantiles).to_frame().reset_index()



,index,Distancia_final
0,0.00,0.000003
1,0.05,0.000096
2,0.10,0.002978
3,0.15,0.004418
4,0.20,0.006006
5,0.25,0.006340
6,0.30,0.006467
7,0.35,0.006911
8,0.40,0.007621
9,0.45,0.007621


In [17]:
pd.options.display.float_format = '{:.6f}'.format
ofertas_final_v2[(ofertas_final_v2["ZONA"] == "URBANA_APROX") & (ofertas_final_v2["FUENTE"] == "R1_MATRICULA")]["Distancia_final"].quantile(quantiles).to_frame().reset_index()

,index,Distancia_final
0,0.000000,0.000000
1,0.050000,0.000002
2,0.100000,0.000005
3,0.150000,0.000008
4,0.200000,0.000009
5,0.250000,0.000013
6,0.300000,0.000019
7,0.350000,0.000026
8,0.400000,0.000037
9,0.450000,0.000045


In [18]:
ofertas_final_v2.loc[ofertas_final_v2["NPN_R1"].notnull(), "CODIGO_PREDIAL"] = ofertas_final_v2.loc[ofertas_final_v2["NPN_R1"].notnull(), "NPN_R1"]
#ofertas_final_v2.loc[ofertas_final_v2["NPN_R1"].notnull(), "Fuente"] = "OIC"
ofertas_final_v2.loc[(ofertas_final_v2["NPN_R1"].isnull()) & (ofertas_final_v2["CODIGO_PREDIAL"].notnull()), "FUENTE"] = "Subdireccion"

ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.000092) & (ofertas_final_v2["ZONA"] == "URBANA_APROX"), "TIPO_GEORREF"] = "APROX_OK"
ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.008925) & (ofertas_final_v2["ZONA"] == "RURAL_APPROX"), "TIPO_GEORREF"] = "APROX_OK"

# ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.000045) & (ofertas_final_v2["ZONA"] == "URBANA_APROX"), "TIPO_GEORREF"] = "APROX_OK"
# ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.007404) & (ofertas_final_v2["ZONA"] == "RURAL_APPROX"), "TIPO_GEORREF"] = "APROX_OK"


In [19]:
ofertas_final_v2[(ofertas_final_v2["CODIGO_PREDIAL"] == "257400000000000000000000000000") & (ofertas_final_v2["TIPO_GEORREF"] == "EXACTA")][["Direccion2"]]

#ofertas_final_v2["CODIGO_PREDIAL"].value_counts()

,Direccion2
20418,"DIAGONAL 38 19 82, SOACHA, CUNDINAMARCA"
24738,"AVENIDA TERREROS CALLE 38 1 50, SOACHA, CUNDIN..."
91576,"CL 38 21 81, SOACHA, CUNDINAMARCA"
91887,"AV TERREROS CL 38 1 50, SOACHA, CUNDINAMARCA"
92250,"CR 21 38 40, SOACHA, CUNDINAMARCA"
...,...
431559,"AV TERREROS CL 38 1 50, SOACHA, CUNDINAMARCA"
437647,"DG 38 19 82, SOACHA, CUNDINAMARCA"
479158,"AVENIDA TERREROS CALLE 38 1 50, SOACHA, CUNDIN..."
483693,"AVENIDA TERREROS CL 38 1 50, SOACHA, CUNDINAMARCA"


In [27]:
os.listdir()
ofertas_final_v2["SUBSTRING_67"] = ofertas_final_v2["CODIGO_PREDIAL"].str[5:7]

# Define the conditions and corresponding values
conditions = [
    ofertas_final_v2['SUBSTRING_67'] == "00",
    ofertas_final_v2['SUBSTRING_67'] == "01",
    (~ofertas_final_v2['SUBSTRING_67'].isin(["01", "00"])) & (ofertas_final_v2['SUBSTRING_67'].notnull())
]
choices = ['RURAL', 'URBANO', 'CENTRO_P']

# Apply the case when statement using numpy.select()
ofertas_final_v2['ZONA'] = np.select(conditions, choices, default='SIN_ASIGNAR')

#ofertas_final_v2.loc[(ofertas_final_v2["FUENTE"] == "R1_MATRICULA") & ~(ofertas_final_v2["ZONA"].isin(["RURAL", "URBANO"])), "ZONA"] = ofertas_final_v2.loc[(ofertas_final_v2["FUENTE"] == "R1_MATRICULA") & ~(ofertas_final_v2["ZONA"].isin(["RURAL", "URBANO"])), "ZONA_2"]

#ofertas_final_v2['ZONA_2'] = ofertas_final_v2['SUBSTRING_67'].apply(lambda x: 'RURAL' if x == '00' else 'URBANO')


#ofertas_final_v2.drop(["SUBSTRING_67", "ZONA_2"], axis = 1, inplace = True)

#ofertas_final_v2.drop((['address', 'CODIGO_ANT_URBANO', 'CODIGO_ANT_RURAL_APROX', 'CODIGO_ANT_URBANO_APROX']), axis = 1, inplace = True)
ofertas_final_v2.to_csv("../output/230713_BASE_PROCESADA_AVALUOS_IGAC.csv")

In [23]:
ofertas_final_v2[["TIPO_GEORREF", "FUENTE"]].value_counts(dropna = False).to_frame()

,,count
TIPO_GEORREF,FUENTE,
NaN,NaN,318417
APROX,Subdireccion,143490
NaN,R1_MATRICULA,12177
EXACTA,Subdireccion,12174
APROX_OK,Subdireccion,11129
APROX,R1_MATRICULA,8277
EXACTA,R1_MATRICULA,6901
APROX_OK,R1_MATRICULA,5673


In [24]:
ofertas_final_v2[["TIPO_GEORREF", "FUENTE"]].value_counts(dropna = False).to_frame()

,,count
TIPO_GEORREF,FUENTE,
NaN,NaN,310563
APROX,Subdireccion,140709
NaN,R1_MATRICULA,12177
EXACTA,Subdireccion,11404
APROX,R1_MATRICULA,9525
APROX_OK,Subdireccion,8193
EXACTA,R1_MATRICULA,6953
APROX_OK,R1_MATRICULA,4065
EXACTA,NaN,19


In [28]:
ofertas_final_v2[["TIPO_GEORREF", "FUENTE", "ZONA"]].value_counts(dropna = False).to_frame()

count
TIPO_GEORREF FUENTE       ZONA               
NaN          NaN          SIN_ASIGNAR  318417
APROX        Subdireccion RURAL        133756
APROX_OK     Subdireccion URBANO        10685
EXACTA       Subdireccion URBANO         9924
APROX        Subdireccion URBANO         9459
NaN          R1_MATRICULA RURAL          6909
                          URBANO         5130
APROX        R1_MATRICULA RURAL          5054
EXACTA       R1_MATRICULA URBANO         4777
APROX_OK     R1_MATRICULA URBANO         4468
APROX        R1_MATRICULA URBANO         3149
EXACTA       R1_MATRICULA RURAL          1992
             Subdireccion RURAL          1644
APROX_OK     R1_MATRICULA RURAL          1148
EXACTA       Subdireccion CENTRO_P        606
APROX_OK     Subdireccion CENTRO_P        442
APROX        Subdireccion CENTRO_P        275
NaN          R1_MATRICULA CENTRO_P        138
EXACTA       R1_MATRICULA CENTRO_P        132
APROX        R1_MATRICULA CENTRO_P         74
APROX_OK     R1_MATRICULA CENTRO_P         57
             Subdireccion RURAL             2

In [147]:
df_cruce_ofertas_R1[df_cruce_ofertas_R1["ZONA"] == "RURAL_APROX"]["Distancia_final"].quantile(quantiles)



0.05     0.000052
0.10     0.004176
0.20     0.006190
0.25     0.008511
0.50     0.035985
0.75     0.064048
0.90    24.048988
0.95    28.122124
Name: Distancia_final, dtype: float64

In [148]:
df_cruce_ofertas_R1[df_cruce_ofertas_R1["ZONA"] == "URBANA_APROX"]["Distancia_final"].quantile(quantiles)

0.05    0.000005
0.10    0.000009
0.20    0.000019
0.25    0.000023
0.50    0.000043
0.75    0.000074
0.90    0.003017
0.95    0.006631
Name: Distancia_final, dtype: float64

In [165]:
df_cruce_ofertas_R1[["TIPO_GEORREF", "Fuente"]].value_counts(dropna = False).to_frame()

count
TIPO_GEORREF Fuente              
APROX        Subdireccion  242928
NaN          NaN           186099
APROX_OK     Subdireccion   41139
EXACTA       Subdireccion   22083
             OIC             5391
APROX_OK     OIC             4136
NaN          OIC             2757
APROX        OIC             1225
EXACTA       NaN               96

In [166]:
df_cruce_ofertas_R1[["TIPO_GEORREF", "Fuente", "ZONA"]].value_counts(dropna = False).to_frame()

count
TIPO_GEORREF Fuente       ZONA                
APROX        Subdireccion RURAL_APROX   191233
NaN          NaN          NaN           186099
APROX        Subdireccion URBANA_APROX   51695
APROX_OK     Subdireccion URBANA_APROX   41139
EXACTA       Subdireccion URBANO         20118
             OIC          URBANO          5164
APROX_OK     OIC          URBANA_APROX    4136
NaN          OIC          NaN             2757
EXACTA       Subdireccion RURAL           1965
APROX        OIC          RURAL_APROX      765
                          URBANA_APROX     460
EXACTA       OIC          RURAL            130
                          NaN               97
             NaN          NaN               96